In [1]:
import pickle
import pandas as pd
import numpy as np
from imp import reload
import torch
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from mymodule import VAE_model
from collections import defaultdict

random_state=34

processor = VAE_model.ProcessSMILES()


In [2]:
# all drugs
with open('/tf/notebooks/code_for_pub/smiles_files/smiles_drugcombANDchembl26.pickle','rb') as f:
    a = pickle.load(f)
with open('/tf/notebooks/code_for_pub/smiles_files/smiles_drugcomb_BY_cid_duplicated.pickle','rb') as f:
    b = pickle.load(f)
smiles = a.append(b).drop_duplicates().reset_index(drop=True)

### 16 bit

In [3]:
m = torch.load('/tf/notebooks/code_for_pub/_logs_as_python_files/vae_training_logs/VAE16_model_best.pth.tar')
encoder = VAE_model.MolEncoder(i=140, o=16, c=len(m['charset']))
encoder.load_state_dict(m['encoder'])
encoder.eval()



MolEncoder(
  (conv_1): Sequential(
    (0): Conv1d(140, 9, kernel_size=(9,), stride=(1,))
    (1): SELU(
      (elu): ELU(alpha=1.6732632423543772, inplace=True)
    )
  )
  (conv_2): Sequential(
    (0): Conv1d(9, 9, kernel_size=(9,), stride=(1,))
    (1): SELU(
      (elu): ELU(alpha=1.6732632423543772, inplace=True)
    )
  )
  (conv_3): Sequential(
    (0): Conv1d(9, 10, kernel_size=(11,), stride=(1,))
    (1): SELU(
      (elu): ELU(alpha=1.6732632423543772, inplace=True)
    )
  )
  (dense_1): Sequential(
    (0): Linear(in_features=280, out_features=435, bias=True)
    (1): SELU(
      (elu): ELU(alpha=1.6732632423543772, inplace=True)
    )
  )
  (lmbd): Lambda(
    (z_mean): Linear(in_features=435, out_features=16, bias=True)
    (z_log_var): Linear(in_features=435, out_features=16, bias=True)
  )
)

In [7]:
m['charset']

[' ',
 '#',
 '%',
 '(',
 ')',
 '+',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '=',
 '@',
 'A',
 'B',
 'C',
 'F',
 'G',
 'H',
 'I',
 'M',
 'N',
 'O',
 'P',
 'R',
 'S',
 'T',
 'Z',
 '[',
 '\\',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'g',
 'i',
 'l',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u']

In [4]:
temp = []
for i in b:
    temp.append(processor.one_hot_encode(STRING=i, 
                                         charset=m['charset'], 
                                         size_of_encoding=len(m['charset']), 
                                         max_len_smiles=140, 
                                         dtype=np.uint8))

temp = torch.from_numpy(np.array(temp, dtype=np.uint8))
vae = TensorDataset(temp, torch.zeros(temp.size()[0]))
v_loader = DataLoader(vae, batch_size=1, shuffle=False)

In [5]:
vae_fps = np.zeros((len(b), 16+1), dtype=np.float) # +1 is because we also need the ID coming from the smiles file
with torch.no_grad():
    for cid, (ind, sm) in zip(b.index, enumerate(v_loader)):
        x_var = Variable(sm[0].type(torch.FloatTensor))
        vae_fps[ind] = np.hstack((np.array(cid), encoder(x_var).detach().numpy().reshape(-1)))
        
vae_fps = pd.DataFrame(data=vae_fps)

In [7]:
drugs_name = '/tf/notebooks/code_for_pub/smiles_files/drugcomb_drugs_export_OCT2020.csv'
drugs = pd.read_csv(drugs_name, names=['dname','id', 'smiles', 'cid'], header=0) # oct2020 version

mapping = defaultdict(list) 
for i in drugs.itertuples(): # map cid to id
    mapping[i.cid] = i.id
    
vae_fps.iloc[:,0] = vae_fps.iloc[:,0].astype(int)
vae_fps.rename(columns={0:'cid'}, inplace=True)
vae_fps['cid'] = vae_fps['cid'].map(mapping)
vae_fps.rename(columns={'cid':'id'}, inplace=True)
vae_fps.set_index(keys='id', drop=True, inplace=True)

In [9]:
vae_fps.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
id,,,,,,,,,,,,,,,,
1,0.163736,0.081561,0.142728,-0.198636,0.167561,-0.578188,0.104223,0.823720,-0.217764,0.277563,0.022141,-0.529946,-0.047073,-0.060431,-0.224169,-0.045983
2,0.065604,-0.603240,0.139630,-0.218917,-0.064989,-0.354771,-0.095955,0.426336,-0.203644,0.198090,0.368956,-0.192502,0.326586,0.396974,0.166425,-0.506837
3,0.068589,0.062158,-0.488432,0.119903,-0.309707,0.420289,-0.203127,-0.369824,0.460347,-0.107599,-0.049481,-0.025287,0.116868,0.182888,0.350447,0.073835
4,-0.314419,0.373166,-0.159299,0.401706,-0.047756,0.143608,0.156649,-0.202489,0.213929,0.038915,-0.545129,0.088216,0.311570,-0.157387,0.122732,-0.243036
5,-0.006526,-0.146289,-0.324931,0.016936,-0.109024,-0.110659,0.102276,0.233056,0.482240,-0.336784,0.223348,0.165616,-0.348579,0.388333,-0.525754,-0.255213


In [15]:
with open('/tf/notebooks/code_for_pub/fp_files/fps_VAE_16bit_new.pickle','wb') as f:
    pickle.dump(vae_fps, f)

### 256 bit

In [10]:
m = torch.load('/tf/notebooks/code_for_pub/_logs_as_python_files/vae_training_logs/VAE256_model_best.pth.tar')
encoder = VAE_model.MolEncoder(i=140, o=256, c=len(m['charset']))
encoder.load_state_dict(m['encoder'])
encoder.eval()



MolEncoder(
  (conv_1): Sequential(
    (0): Conv1d(140, 9, kernel_size=(9,), stride=(1,))
    (1): SELU(
      (elu): ELU(alpha=1.6732632423543772, inplace=True)
    )
  )
  (conv_2): Sequential(
    (0): Conv1d(9, 9, kernel_size=(9,), stride=(1,))
    (1): SELU(
      (elu): ELU(alpha=1.6732632423543772, inplace=True)
    )
  )
  (conv_3): Sequential(
    (0): Conv1d(9, 10, kernel_size=(11,), stride=(1,))
    (1): SELU(
      (elu): ELU(alpha=1.6732632423543772, inplace=True)
    )
  )
  (dense_1): Sequential(
    (0): Linear(in_features=280, out_features=435, bias=True)
    (1): SELU(
      (elu): ELU(alpha=1.6732632423543772, inplace=True)
    )
  )
  (lmbd): Lambda(
    (z_mean): Linear(in_features=435, out_features=256, bias=True)
    (z_log_var): Linear(in_features=435, out_features=256, bias=True)
  )
)

In [11]:
temp = []
for i in b:
    temp.append(processor.one_hot_encode(STRING=i, 
                                         charset=m['charset'], 
                                         size_of_encoding=len(m['charset']), 
                                         max_len_smiles=140, 
                                         dtype=np.uint8))

temp = torch.from_numpy(np.array(temp, dtype=np.uint8))
vae = TensorDataset(temp, torch.zeros(temp.size()[0]))
v_loader = DataLoader(vae, batch_size=1, shuffle=False)

In [12]:
vae_fps256 = np.zeros((len(b), 256+1), dtype=np.float) # +1 is because we also need the ID coming from the smiles file
with torch.no_grad():
    for cid, (ind, sm) in zip(b.index, enumerate(v_loader)):
        x_var = Variable(sm[0].type(torch.FloatTensor))
        vae_fps256[ind] = np.hstack((np.array(cid), encoder(x_var).detach().numpy().reshape(-1)))
        
vae_fps256 = pd.DataFrame(data=vae_fps256)

In [13]:
drugs_name = '/tf/notebooks/code_for_pub/smiles_files/drugcomb_drugs_export_OCT2020.csv'
drugs = pd.read_csv(drugs_name, names=['dname','id', 'smiles', 'cid'], header=0) # oct2020 version

mapping = defaultdict(list) 
for i in drugs.itertuples(): # map cid to id
    mapping[i.cid] = i.id
    
vae_fps256.iloc[:,0] = vae_fps256.iloc[:,0].astype(int)
vae_fps256.rename(columns={0:'cid'}, inplace=True)
vae_fps256['cid'] = vae_fps256['cid'].map(mapping)
vae_fps256.rename(columns={'cid':'id'}, inplace=True)
vae_fps256.set_index(keys='id', drop=True, inplace=True)

In [14]:
vae_fps256.head()

,1,2,3,4,5,6,7,8,9,10,...,247,248,249,250,251,252,253,254,255,256
id,,,,,,,,,,,,,,,,,,,,,
1,0.037148,0.025033,-0.036369,0.102345,-0.093183,0.059088,-0.090992,0.088404,-0.003136,-0.020551,...,-0.105050,0.020967,-0.045898,0.049329,-0.058503,-0.058598,-0.019961,0.118273,0.050400,-0.091401
2,0.038856,-0.005299,-0.050357,-0.033386,-0.029205,-0.031847,0.065195,0.182351,0.073959,-0.060510,...,-0.014917,0.109559,-0.075734,-0.000794,-0.069747,0.061757,-0.018116,0.052775,0.003630,0.127593
3,-0.213672,0.049369,0.154992,-0.017447,0.065578,0.048477,0.066674,-0.051494,0.233828,-0.014774,...,0.180679,-0.158507,0.064181,0.061846,0.058301,-0.019160,0.073533,-0.073078,-0.041435,-0.016401
4,0.034581,-0.048522,0.009862,0.073342,-0.117260,0.023983,0.078561,-0.083111,-0.100042,-0.060413,...,-0.000272,-0.110841,-0.000027,0.126634,-0.027044,-0.038833,0.001398,0.020722,-0.023089,-0.138941
5,0.060897,-0.024376,0.014705,0.004875,-0.083799,-0.103723,0.044524,-0.150909,0.033970,0.037664,...,0.110754,0.059234,0.050372,0.030529,-0.052521,-0.081916,0.007758,-0.008155,-0.041844,0.141539


In [16]:
with open('/tf/notebooks/code_for_pub/fp_files/fps_VAE_256bit_new.pickle','wb') as f:
    pickle.dump(vae_fps256, f)